In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random
from playlist import *
from collections import defaultdict
from numpy.random import choice
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import random
import playlist
reload(playlist)

# client_credentials_manager = SpotifyClientCredentials()
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# results = sp.search(q='weezer', limit=20)
# for i, t in enumerate(results['tracks']['items']):
#     print(' ', i, t['name'])

In [ ]:
playlist_list = playlist.get_playlist()

In [ ]:
len(playlist_list)

In [ ]:
import sys
sys.getsizeof(playlist_list)

In [ ]:
# def get_tracks(playlist):
#     track_list = set([])
#     for i in range(playlist['num_tracks']):
#         track_list.add(playlist['tracks'][i]['track_uri'])
#     return track_list

In [ ]:
# track_audio_features = {}
# for i in range (len(playlist_list)):
#         playlist = playlist_list[i]
#         for j in range(playlist['num_tracks']):
#             track_uri = playlist['tracks'][j]['track_uri']
#             if track_uri not in track_audio_features:
#                 track_audio_features[track_uri] = sp.audio_features([track_uri])[0]

In [ ]:
# import gzip
# import pickle

# with gzip.open('audio_features_1K.pickle.gz','wb') as output:
#     pickle.dump(track_audio_features ,output)

In [ ]:
# import gzip
# import pickle

# with gzip.open("audio_features_1K_p2.pickle.gz", "rb") as inp:
#     track_audio_features = pickle.load(inp)

In [ ]:
%%time
audio_df = pd.read_csv('playlist-tracks.tracksFeatureCache.csv', index_col = 'uri')
# audio_df.head()

In [ ]:
audio_df = audio_df[['acousticness', 'danceability', 'duration_ms', 'energy',
        'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature',
       'valence']]
audio_df = audio_df.astype(float)
audio_df.head()

In [ ]:

# sp.audio_features(['spotify:track:20ORwCJusz4KS2PbTPVNKo'])

In [ ]:
# sp.audio_analysis('spotify:track:20ORwCJusz4KS2PbTPVNKo')

In [ ]:
# sp.artist('spotify:artist:2Hjj68yyUPiC0HKEOigcEp')

In [ ]:
# audio_df = pd.DataFrame(track_audio_features).T
# audio_df = audio_df[['acousticness', 'danceability', 'duration_ms', 'energy',
#         'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
#        'speechiness', 'tempo', 'time_signature',
#        'valence']]
# audio_df = audio_df.astype(float)

In [ ]:
audio_df.shape

In [ ]:
audio_df.head()

In [ ]:
audio_df = audio_df.fillna(0)

In [ ]:
# normalised panda data frame
from sklearn import preprocessing

x = audio_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
new_audio_df = pd.DataFrame(x_scaled)
new_audio_df.index = audio_df.index
audio_df = new_audio_df
# new_audio_df.head()

In [ ]:
def get_track_embedding(track_uri):
    return audio_df.loc[track_uri].values

In [ ]:
get_track_embedding('spotify:track:0uqPG793dkDDN7sCUJJIVC').shape

In [ ]:
cached_playlist_embedding = {}
num_missing_tracks = 0

def get_playlist_embedding(playlist_id):
    global num_missing_tracks
    # check if already in cache 
    if playlist_id in cached_playlist_embedding:
        return cached_playlist_embedding[playlist_id]

    embedding_list = []
    for i in range(playlist_list[playlist_id]['num_tracks']):
        track_uri = playlist_list[playlist_id]['tracks'][i]['track_uri']
    #     print(type(track_df.loc[track_uri]))
        if track_uri in audio_df.index:
            embedding_list.append(get_track_embedding(track_uri))
        else:
            num_missing_tracks += 1
    embedding = np.mean(embedding_list, axis = 0)
    embedding = embedding.reshape(1, -1)
    norm_embedding = np.linalg.norm(embedding)
    # put in the cache
    cached_playlist_embedding[playlist_id] = (embedding, embedding/norm_embedding)
    return embedding

In [ ]:
for playlist_id in tqdm(range(len(playlist_list))):
    get_playlist_embedding(playlist_id)


In [ ]:
print num_missing_tracks

In [ ]:
_dot = np.inner
_linear_cache = tuple([get_playlist_embedding(playlist_id)
                       for playlist_id in range(len(playlist_list))])

def cal_similarity(playlist_id_1, playlist_id_2):
#     a, norm_a = get_playlist_embedding(playlist_id_1)
#     b, norm_b = get_playlist_embedding(playlist_id_2)
    _, normed_a = _linear_cache[playlist_id_1]
    _, normed_b = _linear_cache[playlist_id_2]
    
    X = _dot(normed_a, normed_b)
#     X = _dot(a, b) / (norm_a * norm_b)
    return X.item()

# 10x slower
def cal_similarity2(playlist_id_1, playlist_id_2):
    similarity = sklearn.metrics.pairwise.cosine_similarity(get_playlist_embedding(playlist_id_1)[0], 
                                                            get_playlist_embedding(playlist_id_2)[0])
#     similarity = sklearn.metrics.pairwise.euclidean_distances(get_playlist_embedding(playlist_id_1).reshape(1, -1), 
#                                                                 get_playlist_embedding(playlist_id_2).reshape(1, -1))
#         return similarity[0][0]
    return similarity[0][0]

In [ ]:
# print(cal_similarity(0, 1))
# print(cal_similarity2(0, 1))

In [ ]:
%%timeit
cal_similarity(0, 1)

In [ ]:
%%timeit
cal_similarity2(0, 1)

In [ ]:
# see https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
def slow_softmax_sample(x):
    # but does not change the answer
    x = np.exp(x - np.max(x))
    prob_dis = x / float(sum(x))
    return int(choice(range(len(x)), 1, p=prob_dis))

# https://timvieira.github.io/blog/post/2014/07/31/gumbel-max-trick/
def softmax_sample(x):
    cdf = np.exp(x - x.max()).cumsum()
    z = cdf[-1]
    u = np.random.uniform(0,1)
    return cdf.searchsorted(u * z)

def fast_softmax_sample(x):
    z = np.random.gumbel(loc=0, scale=1, size=x.shape)
    return (x + z).argmax(axis=0)

In [ ]:
%%timeit
slow_softmax_sample(np.array([1,2,3]))

In [ ]:
%%timeit 
softmax_sample(np.array([1,2,3]))

In [ ]:
%%timeit 
fast_softmax_sample(np.array([1,2,3]))

In [ ]:
def biased_audio_random_walk_playlist(query,g,dropped_track_id, beta = 1000):
    
    N = 10000
    totSteps = 0
    nHighVisited = 0
    nt = 200
    nv = 5
    num_visits = defaultdict(int)

    weight_cache = {}
    def cal_weight(edge):
        if edge in weight_cache:
            return weight_cache[edge]

        val = cal_similarity(query,edge)
        weight_cache[edge] = val
        return val

    while totSteps < N and nHighVisited < nt: 
        currPlaylist = query
        # Number of iterations
        currSteps = 5
        for i in range(currSteps):
            # takes 1 step (from a playlist to track)
            edges = list(g[currPlaylist])
            if currPlaylist == query:
                edges.remove(dropped_track_id)
            currTrack = random.choice(edges)
            # takes 1 step (from a track to playlist)
            edges = list(g[currTrack])
            if currTrack == dropped_track_id:
                edges.remove(query)
#             weights = [f(edge) for edge in edges]
            weights = [cal_weight(edge) for edge in edges]
            weights_np = np.asarray(weights)

# see https://stackoverflow.com/questions/34968722/how-to-implement-the-softmax-function-in-python
#           # this bit makes it more stable numerically
            max_weight = np.max(weights_np)
            weights_np -= max_weight
            # but does not change the answer
            weights_np = np.exp(beta * weights_np)

            prob_dis = weights_np / float(sum(weights_np))
            currPlaylist = int(choice(edges, 1, p=prob_dis))
            num_visits[currTrack] += 1
            if num_visits[currTrack] == nv:
                nHighVisited += 1
        totSteps += currSteps
    return num_visits

In [ ]:
g, tracks_dict, tracks_id_dict = playlist.make_graph_dict(playlist_list)

In [ ]:
def cal_most_visited_playlist(playlist_list, playlist_id):
    dropped_track_id = random.choice(tuple(g[playlist_id]))

    if len(g[playlist_id]) < 5:
        return None, None
    num_visits = biased_audio_random_walk_playlist(playlist_id, g, dropped_track_id)
    suggested_ids = sorted(num_visits, key=num_visits.get, reverse=True)
    return dropped_track_id, suggested_ids

In [ ]:
# %%prun
# cal_most_visited_playlist(playlist_list, 0)

In [ ]:
# all_true_ids, all_suggested_ids = cal_most_visited(playlist_list)
all_true_ids, all_suggested_ids = playlist.process_playlists(cal_most_visited_playlist,
                                                             playlist_list, multiprocess=False)

In [ ]:
# np.savez_compressed('audio_featured_biased_100k_500_result.npz',all_true_ids = all_true_ids, 
#                    all_suggested_ids = all_suggested_ids)

In [ ]:
results = playlist.cal_results(playlist_list, all_true_ids, all_suggested_ids)
filename = 'results%d_audio_featured_biased_random_walk_10K.txt' % (len(playlist_list),)
with open(filename, 'w') as output:
    output.write(str(results))

In [ ]:

# np.any(audio_df.isna(), axis = 0)

In [ ]:
# help(audio_df.fillna)

In [ ]:
import matplotlib.pyplot as plt

weights = [cal_similarity(0, 1+i) for i in range(10)]
weights_np = np.asarray(weights)
prob_dis = weights_np / float(sum(weights_np))
plt.plot(prob_dis, label="flat")
for beta in [1,2,5,10,50]:
    weights_np = np.asarray(weights)
    # this bit makes it more stable numerically
    max_weight = np.max(weights_np)
    weights_np -= max_weight
    # but does not change the answer
    weights_np = np.exp(beta * weights_np)

    prob_dis = weights_np / float(sum(weights_np))
    plt.plot(prob_dis, label="beta=" + str(beta))
plt.legend()
plt.show()

In [ ]:
weights